# imports

In [1]:
!pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00


In [22]:
from datasets import load_dataset
import numpy as np
from model import MemNN

dataset = load_dataset("facebook/babi_qa", "en-10k-qa1")

  0%|          | 0/2 [00:00<?, ?it/s]

# Preprocess data

In [3]:
def get_dataset(dataset, max_story_length=10):

    # Initialize story and question arrays
    stories = [[] for _ in range(max_story_length)]
    questions = [[] for _ in range(max_story_length)]
    answers = [[] for _ in range(max_story_length)]

    for i in range(len(dataset["train"]["story"])):
        data = dataset["train"]["story"][i]["text"]
        current_ans = dataset["train"]["story"][i]["answer"]
        current_story = []  # Reset the current story for each batch
        current_questions = []  # Reset the current questions for each batch

        for indx, sentence in enumerate(data):
            if "?" in sentence:
                # Determine the length of the current story
                story_length = len(current_story)

                # Append the current story and question to the respective arrays
                if story_length <= max_story_length:
                    stories[story_length - 1].append(list(current_story))
                    questions[story_length - 1].append(sentence)
                    answers[story_length - 1].append(current_ans[indx])
            else:
                current_story.append(sentence)



    # Convert the lists to numpy arrays
    stories = [np.array(story) for story in stories]
    questions = [np.array(question).reshape(-1,1) for question in questions]
    answers = [np.array(answer) for answer in answers]

    return stories, questions, answers


In [4]:
stories, questions, answers = get_dataset(dataset)

In [5]:
questions[1].shape, stories[1].shape

((2000, 1), (2000, 2))

In [6]:
max_story_len = 10

def pad_stories(stories, max_story_len=10):

    batch_size, n_sentence = stories.shape

    if n_sentence < max_story_len:
        padding = np.array([[""] * (max_story_len - n_sentence)] * batch_size)
        stories = np.concatenate((stories, padding), axis=1)

    return stories

merged_stories = np.array([pad_stories(stories[i],max_story_len) for i in range(1,11,2) ]).reshape(-1,max_story_len)
merged_questions = np.array([questions[i] for i in range(1,11,2)]).reshape(-1,1)
merged_answers = np.array([answers[i] for i in range(1,11,2)]).reshape(-1,1)

In [7]:
indx = 16

merged_stories[indx], merged_questions[indx], merged_answers[indx]

(array(['Mary journeyed to the garden.', 'Sandra went back to the office.',
        '', '', '', '', '', '', '', ''], dtype='<U33'),
 array(['Where is Sandra?'], dtype='<U16'),
 array(['office'], dtype='<U8'))

In [8]:
merged_stories.shape, merged_questions.shape, merged_answers.shape

((10000, 10), (10000, 1), (10000, 1))

In [23]:
stories[-1].shape

(2000, 10)

In [10]:
vocab = set()
for s in stories[-1]:
    s = " ".join(s)
    for w in s.split():
        if "." in w:
            w = w[:-1]
        vocab.add(w)
for q in questions[-1]:
    q = q[0]
    for w in q.split():
        if "?" in w:
            w = w[:-1]
        vocab.add(w)

vocab = np.array(list(vocab))


def get_vocab(stories, questions, answers):
    pass

In [11]:
len(vocab)

19

# MeMNet

In [24]:
mem_1 = MemNN(vocab, max_query_len=1, max_story_len=10, vocab_size=1000, embedding_size=128, k=5)
mem_1.batch_size = 32

In [25]:
mem_1.compile()

In [26]:
mem_1.model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 story (InputLayer)             [(32, 10, 128)]      0           []                               
                                                                                                  
 query (InputLayer)             [(32, 1, 128)]       0           []                               
                                                                                                  
 lstm_4 (LSTM)                  (32, 10, 128)        131584      ['story[0][0]']                  
                                                                                                  
 tf.math.l2_normalize_13 (TFOpL  (32, 1, 128)        0           ['query[0][0]']                  
 ambda)                                                                                     

In [27]:
max_story_len = 2

mem_1.fit(stories[max_story_len-1], questions[max_story_len-1].reshape(-1,1), answers[max_story_len-1], epochs=50, validation_split=0.2)

Epoch 1/50
50/50 [==============================] - 6s 39ms/step - loss: 2.5247 - accuracy: 0.1606 - val_loss: 2.0615 - val_accuracy: 0.1800
Epoch 2/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7895 - accuracy: 0.3150 - val_loss: 1.5931 - val_accuracy: 0.3975
Epoch 3/50
50/50 [==============================] - 1s 19ms/step - loss: 1.4812 - accuracy: 0.4400 - val_loss: 1.3475 - val_accuracy: 0.4400
Epoch 4/50
50/50 [==============================] - 1s 23ms/step - loss: 1.3284 - accuracy: 0.4850 - val_loss: 1.2486 - val_accuracy: 0.5075
Epoch 5/50
50/50 [==============================] - 2s 31ms/step - loss: 1.2172 - accuracy: 0.5256 - val_loss: 1.1386 - val_accuracy: 0.5350
Epoch 6/50
50/50 [==============================] - 1s 20ms/step - loss: 1.1599 - accuracy: 0.5200 - val_loss: 1.0600 - val_accuracy: 0.5450
Epoch 7/50
50/50 [==============================] - 1s 22ms/step - loss: 1.0549 - accuracy: 0.5462 - val_loss: 1.0271 - val_accuracy: 0.5300
Epoch 8/50
50

In [28]:
max_story_len = 4

mem_2 = MemNN(vocab, max_query_len=1, max_story_len=10, vocab_size=1000, embedding_size=128, k=5)
mem_2.batch_size = 32

mem_2.compile()

mem_2.fit(stories[max_story_len-1], questions[max_story_len-1].reshape(-1,1), answers[max_story_len-1], epochs=50, validation_split=0.2)

Epoch 1/50
50/50 [==============================] - 7s 40ms/step - loss: 2.3089 - accuracy: 0.1425 - val_loss: 1.8308 - val_accuracy: 0.1875
Epoch 2/50
50/50 [==============================] - 1s 18ms/step - loss: 1.8075 - accuracy: 0.1838 - val_loss: 1.7922 - val_accuracy: 0.1775
Epoch 3/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7355 - accuracy: 0.2612 - val_loss: 1.6978 - val_accuracy: 0.3175
Epoch 4/50
50/50 [==============================] - 1s 20ms/step - loss: 1.6537 - accuracy: 0.3269 - val_loss: 1.6328 - val_accuracy: 0.3125
Epoch 5/50
50/50 [==============================] - 1s 19ms/step - loss: 1.6271 - accuracy: 0.3150 - val_loss: 1.6502 - val_accuracy: 0.2850
Epoch 6/50
50/50 [==============================] - 1s 19ms/step - loss: 1.6083 - accuracy: 0.3394 - val_loss: 1.6328 - val_accuracy: 0.3125
Epoch 7/50
50/50 [==============================] - 1s 19ms/step - loss: 1.6061 - accuracy: 0.3269 - val_loss: 1.6218 - val_accuracy: 0.3125
Epoch 8/50
50

In [29]:
max_story_len = 6

mem_3 = MemNN(vocab, max_query_len=1, max_story_len=10, vocab_size=1000, embedding_size=128, k=5)
mem_3.batch_size = 32

mem_3.compile()

mem_3.fit(stories[max_story_len-1], questions[max_story_len-1].reshape(-1,1), answers[max_story_len-1], epochs=50, validation_split=0.2)

Epoch 1/50
50/50 [==============================] - 7s 44ms/step - loss: 2.4383 - accuracy: 0.1700 - val_loss: 1.9516 - val_accuracy: 0.1600
Epoch 2/50
50/50 [==============================] - 1s 20ms/step - loss: 1.8543 - accuracy: 0.1800 - val_loss: 1.8057 - val_accuracy: 0.1825
Epoch 3/50
50/50 [==============================] - 1s 18ms/step - loss: 1.8130 - accuracy: 0.1713 - val_loss: 1.8133 - val_accuracy: 0.2050
Epoch 4/50
50/50 [==============================] - 1s 20ms/step - loss: 1.7873 - accuracy: 0.2025 - val_loss: 1.8053 - val_accuracy: 0.1650
Epoch 5/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7754 - accuracy: 0.2094 - val_loss: 1.7719 - val_accuracy: 0.2200
Epoch 6/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7871 - accuracy: 0.2219 - val_loss: 1.8170 - val_accuracy: 0.1600
Epoch 7/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7827 - accuracy: 0.1956 - val_loss: 1.7950 - val_accuracy: 0.1675
Epoch 8/50
50

In [ ]:
# merged dataset
max_story_len = 6

mem_4 = MemNN(vocab, max_query_len=1, max_story_len=10, vocab_size=1000, embedding_size=128, k=5)
mem_4.batch_size = 32

mem_4.compile()

mem_4.fit(stories[max_story_len-1], questions[max_story_len-1].reshape(-1,1), answers[max_story_len-1], epochs=50, validation_split=0.2)


mem_4.fit(merged_stories, merged_questions, merged_answers, epochs=150, validation_split=0.2, shuffle=True)

Epoch 1/50
50/50 [==============================] - 7s 54ms/step - loss: 2.3942 - accuracy: 0.0913 - val_loss: 1.9258 - val_accuracy: 0.1600
Epoch 2/50
50/50 [==============================] - 1s 19ms/step - loss: 1.8424 - accuracy: 0.1750 - val_loss: 1.8109 - val_accuracy: 0.1750
Epoch 3/50
50/50 [==============================] - 1s 20ms/step - loss: 1.8216 - accuracy: 0.1681 - val_loss: 1.8086 - val_accuracy: 0.1600
Epoch 4/50
50/50 [==============================] - 1s 19ms/step - loss: 1.8004 - accuracy: 0.1825 - val_loss: 1.8004 - val_accuracy: 0.1800
Epoch 5/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7992 - accuracy: 0.1831 - val_loss: 1.8052 - val_accuracy: 0.1775
Epoch 6/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7959 - accuracy: 0.1850 - val_loss: 1.8205 - val_accuracy: 0.1450
Epoch 7/50
50/50 [==============================] - 1s 19ms/step - loss: 1.7898 - accuracy: 0.2062 - val_loss: 1.8071 - val_accuracy: 0.1725
Epoch 8/50
50